In [13]:
import pandas as pd
import os
from datetime import datetime

In [35]:
# -- years being used for evaluation
Years = ['2014']

# -- spatial key being used for calculation
SpatialKey = 'GEOID'

# -- use Fiscal or Calendar year
UseFiscalYear = False
if UseFiscalYear == True:
    YearType = 'FiscalYr'
else:
    YearType = 'CalendarYr'
# -- input path
ParkDataInputPath = '../../shapeData/Tables/ParkQualityScores/'

# -- output path
WriteOutputPath = '../../shapeData/Tables/AllAttributes/'

In [39]:
# -- Bring In external Data
if SpatialKey == 'ZIPCODE':
    # -- Set Date
    Date = '07-13-2015'
    # -- Input Path
    DataInputPath = '../../shapeData/Tables/ZipcodeExternalData/'
    # -- Bring in the socioeconomic Data
    ZipcodesocioEconomicData = pd.read_csv(DataInputPath+'ZipCodeExternalDataSummary.csv')
    # -- Bring in the market Value Data
    medianMarketValueZipCode = pd.read_csv(DataInputPath+'medianMarketValueZipCode.csv')
    medianMarketValueZipCode = medianMarketValueZipCode[['ZipCode','Market_Val']]
    # -- Bring in the economic Data & clean
    economicDataZipCode = pd.read_csv(DataInputPath+'zbp13totals.csv')
    economicDataZipCode = economicDataZipCode[['zip','emp','ap','est']]
    economicDataZipCode = economicDataZipCode.rename(columns={'zip':'ZipCode','emp':'TotalMidMarchEmployees', 'ap':'TotalAnnualPayroll($1,000)', 'est':'TotalNumberofEstablishments'})
    # -- Bring in Functional Parkland Data
    ZipcodeFunctionalParkland = pd.read_excel(DataInputPath+'CUSP_Adjusted_Spatial_Data.xlsx')
    ZipcodeFunctionalParklandSum = ZipcodeFunctionalParkland.groupby(by=['ZIPCODE'])['Acres_Adj'].sum().reset_index()
    # -- Mereg All data
    ZipCodeMarketValueSocioeconomic = pd.merge(ZipcodesocioEconomicData, medianMarketValueZipCode, right_on='ZipCode', left_on= 'ZipCode5', how='left')
    ZipCodeMarketValueSocioeconomicEconomicData = pd.merge(ZipCodeMarketValueSocioeconomic, economicDataZipCode, on= 'ZipCode', how='left')
    ZipCodeMarketValueSocioeconomicEconomicData['ZIPCODE']=ZipCodeMarketValueSocioeconomicEconomicData['ZipCode5']
    AllExternalData = pd.merge(ZipCodeMarketValueSocioeconomicEconomicData, ZipcodeFunctionalParklandSum, on = 'ZIPCODE', how='left')
    AllExternalData['PopulationPerParkland'] = AllExternalData['TotalPopulation']/AllExternalData['Acres_Adj']
    AllExternalData['EmployeesPerParkland'] = AllExternalData['TotalMidMarchEmployees']/AllExternalData['Acres_Adj']
else:
    # -- Set Date
    Date = '07-14-2015'
    # -- Input Path
    DataInputPath = '../../shapeData/Tables/CensusTractExternalData/'
    # -- Bring in the socioeconomic Data
    censustractsocioEconomicData = pd.read_csv(DataInputPath+'CensusTractExternalDataSummary.csv')
    censustractsocioEconomicData['GEOID'] = censustractsocioEconomicData['GEOID'].astype(str)
    # -- Bring in the market Value Data
    medianMarketValueCensusTract = pd.read_csv(DataInputPath+'medianMarketValueCensusTract.csv')
    medianMarketValueCensusTract = medianMarketValueCensusTract[['CensusTract','Market_Val']]
    # -- Bring in the employess Data 
    employeeCensusTract = pd.read_csv(DataInputPath+'WACemployeesCensusTract2012.csv')
    employeeCensusTract = employeeCensusTract[['GEOID','NumberOfEmployeesWAC']]
    employeeCensusTract['GEOID'] = employeeCensusTract['GEOID'].astype(str)
    # -- Bring in Functional Parkland Data
    CensusTractFunctionalParkland = pd.read_excel(DataInputPath+'Park_Area_Census_Tract.xlsx')
    CensusTractFunctionalParklandSum = CensusTractFunctionalParkland.groupby(by=['GEOID'])['Fractional Acres'].sum().reset_index()
    # -- Merge All data
    CensusTractMarketValueSocioeconomic = pd.merge( medianMarketValueCensusTract,censustractsocioEconomicData, left_on='CensusTract', right_on= 'GEOID', how='left')
    CensusTractMarketValueSocioeconomicEmployee = pd.merge( CensusTractMarketValueSocioeconomic,employeeCensusTract, on = 'GEOID', how = 'left')
    AllExternalData = pd.merge(CensusTractMarketValueSocioeconomicEmployee, CensusTractFunctionalParklandSum, on = 'GEOID', how = 'left')
    AllExternalData['PopulationPerParkland'] = AllExternalData['TotalPopulation']/AllExternalData['Fractional Acres']
    AllExternalData['EmployeesPerParkland'] = AllExternalData['NumberOfEmployeesWAC']/AllExternalData['Fractional Acres']

In [40]:
def ReadScores(Year, Date):
# -- Bring in Park Quality Data and Merge
    if UseFiscalYear:
        parkQuality = pd.read_csv(ParkDataInputPath + 'ParkQuality_%s_%s_%s_%s.csv' % (SpatialKey, Year, YearType, Date), index_col=0)
        if SpatialKey == 'GEOID':
            parkQuality['GEOID'] = parkQuality['GEOID'].astype(str)
    else:
        parkQuality = pd.read_csv(ParkDataInputPath + 'ParkQuality_%s_%s_%s_%s.csv' % (SpatialKey, Year, YearType, Date), index_col=0)
        if SpatialKey == 'GEOID':
            parkQuality['GEOID'] = parkQuality['GEOID'].astype(str)

    return pd.merge(AllExternalData, parkQuality, on= SpatialKey, how='left')

In [41]:
def WriteScores(df, Year):
    # -- Write to csv
    now = datetime.now()
    if UseFiscalYear:
        YearType = 'FiscalYr'
    else:
        YearType = 'CalendarYr'
    df.to_csv(WriteOutputPath + 'AllAttributes_%s_%s_%s_%s.csv' % ( SpatialKey, Year.replace(',','&'), YearType, now.strftime('%m-%d-%Y')), sep=',')

In [42]:
for Year in Years:
    AllAttributes = ReadScores(Year, Date)
    WriteScores(AllAttributes, Year)

In [43]:
AllAttributes

,CensusTract_x,Market_Val,GEOID,NameArea,QualifyingName,AbbreviationUSPS,SummaryLevel,GeographicComponent,FileIdentification,LogicalRecordNumber,...,UnemploymentPercentage,MedianhouseholdIncome,HouseholdsGiniIndex,NumberOfEmployeesWAC,Fractional Acres,PopulationPerParkland,EmployeesPerParkland,Naive Normalized Score,Area Normalized Score,Amenities & Area Normalized Score
0,3600500000nan,121500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36005000100,527664000,36005000100,"Census Tract 1, Bronx County, New York","Census Tract 1, Bronx County, New York",ny,140,0,ACSSF,3473,...,#DIV/0!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36005000200,328000,36005000200,"Census Tract 2, Bronx County, New York","Census Tract 2, Bronx County, New York",ny,140,0,ACSSF,3474,...,8.464655561,69514,0.3944,56,NaN,NaN,NaN,0.937879,0.911987,0.912247
3,36005000400,359000,36005000400,"Census Tract 4, Bronx County, New York","Census Tract 4, Bronx County, New York",ny,140,0,ACSSF,3475,...,5.713271824,73036,0.3710,257,NaN,NaN,NaN,0.926407,0.930381,0.923593
4,36005001600,380000,36005001600,"Census Tract 16, Bronx County, New York","Census Tract 16, Bronx County, New York",ny,140,0,ACSSF,3476,...,6.563307494,30817,0.4847,601,NaN,NaN,NaN,0.979167,0.979167,0.979167
5,36005001900,434000,36005001900,"Census Tract 19, Bronx County, New York","Census Tract 19, Bronx County, New York",ny,140,0,ACSSF,3477,...,18.46153846,33862,0.4566,5750,NaN,NaN,NaN,0.920833,0.912864,0.918128
6,36005002000,390000,36005002000,"Census Tract 20, Bronx County, New York","Census Tract 20, Bronx County, New York",ny,140,0,ACSSF,3478,...,13.50099272,19150,0.5211,122,NaN,NaN,NaN,0.933636,0.945605,0.945745
7,36005002300,799000,36005002300,"Census Tract 23, Bronx County, New York","Census Tract 23, Bronx County, New York",ny,140,0,ACSSF,3479,...,13.29561528,13902,0.4932,18,NaN,NaN,NaN,0.912500,0.881609,0.879951
8,36005002400,535000,36005002400,"Census Tract 24, Bronx County, New York","Census Tract 24, Bronx County, New York",ny,140,0,ACSSF,3480,...,#DIV/0!,NaN,NaN,228,NaN,NaN,NaN,0.889576,0.834835,0.830876
9,36005002500,499000,36005002500,"Census Tract 25, Bronx County, New York","Census Tract 25, Bronx County, New York",ny,140,0,ACSSF,3481,...,12.06804891,14628,0.4985,222,NaN,NaN,NaN,0.812500,0.812500,0.812500
